In [1]:
! git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...


In [12]:
%cd yolov7

C:\Users\Dhriti Shah\Downloads\yolov7


In [13]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt -o yolov7-w6-pose.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  153M    0  767k    0     0   482k      0  0:05:26  0:00:01  0:05:25  834k
  4  153M    4 7756k    0     0  2994k      0  0:00:52  0:00:02  0:00:50 4039k
 11  153M   11 17.5M    0     0  4997k      0  0:00:31  0:00:03  0:00:28 6142k
 18  153M   18 28.8M    0     0  6436k      0  0:00:24  0:00:04  0:00:20 7535k
 26  153M   26 40.3M    0     0  7391k      0  0:00:21  0:00:05  0:00:16 8397k
 34  153M   34 52.7M    0     0  8198k      0  0:00:19  0:00:06  0:00:13 10.4M
 42  153M   42 65.4M    0     0  8829k      0  0:00:17  0:00:07  0:00:10 11.5M
 51  153M   51 78.9M    0     0  9391k      0  0:0

In [2]:
import torch
from torchvision import transforms
import utils
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import os
import matplotlib.pyplot as plt
import cv2
import numpy as np


ModuleNotFoundError: No module named 'utils.datasets'

In [3]:
def load_model():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torch.load("C:/Users/Dhriti Shah/Downloads/yolov7/yolov7-w6-pose.pt", map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model



In [4]:
def run_inference(image_path, model):
    image = cv2.imread(image_path)  # Load the image
    # Resize and pad the image
    image = letterbox(image, 960, stride=64, auto=True)[0]
    # Apply transforms
    image = transforms.ToTensor()(image)
    # Turn the image into a batch
    image = image.unsqueeze(0)
    # Perform pose estimation inference
    output, _ = model(image)
    return output, image


In [18]:
def visualize_output(output, image):
    output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(nimg)
    plt.show()

In [5]:
model = load_model()
image_folder = 'C:/Users/Dhriti Shah/Downloads/archive/var_200/VAR/Fouls'

# Iterate through images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  # Ensure you're processing image files
        image_path = os.path.join(image_folder, filename)
        
        # Run inference on the image
        output, image = run_inference(image_path, model)
        
        # Visualize or save the results as needed
        # You can use functions like output_to_keypoint and plot_skeleton_kpts to visualize the results
        # Or save the results to a file
        
        # Example: Visualize keypoints and skeletons
        visualize_output(output, image)

ModuleNotFoundError: No module named 'models'